In [190]:
import pandas as pd
import numpy as np
from statsmodels.formula.api import ols
import statsmodels
df = pd.read_csv('kc_house_data.csv')

## Changes Made To DataFrame

In [ ]:
## dropped'id' because it's useless lol
## dropped 'date' because its too specific and replaced with a column that shows month categorically
## dropped'yr_built' because it doesn't tell us much BUT I also replaced with this column with how old the home is in 
## years
## dropped'yr_renovated' and made categorical as to whether it has been renovated or not
## dropped 'lat', feels useless
## dropped 'long', feels useless
## made zipcode into a string to make categorical
## made sqft_basement into floats (it was a string)
## created a total_sq_ft column combining basement sq fit with the rest of the house

In [132]:
def make_month(string):
    if string[:2] == '1/':
        return 'Jan'
    if string[:2] == '2/':
        return 'Feb'
    if string[:2] == '3/':
        return 'Mar'
    if string[:2] == '4/':
        return 'Apr'
    if string[:2] == '5/':
        return 'May'
    if string[:2] == '6/':
        return 'Jun'
    if string[:2] == '7/':
        return 'Jul'
    if string[:2] == '8/':
        return 'Aug'
    if string[:2] == '9/':
        return 'Sep'
    if string[:2] == '10':
        return 'Oct'
    if string[:2] == '11':
        return 'Nov'
    if string[:2] == '12':
        return 'Dec'

In [3]:
df.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [4]:
df.head()

id        date     price  bedrooms  bathrooms  sqft_living  \
0  7129300520  10/13/2014  221900.0         3       1.00         1180   
1  6414100192   12/9/2014  538000.0         3       2.25         2570   
2  5631500400   2/25/2015  180000.0         2       1.00          770   
3  2487200875   12/9/2014  604000.0         4       3.00         1960   
4  1954400510   2/18/2015  510000.0         3       2.00         1680   

   sqft_lot  floors  waterfront  view  ...  grade  sqft_above  sqft_basement  \
0      5650     1.0         NaN   0.0  ...      7        1180            0.0   
1      7242     2.0         0.0   0.0  ...      7        2170          400.0   
2     10000     1.0         0.0   0.0  ...      6         770            0.0   
3      5000     1.0         0.0   0.0  ...      7        1050          910.0   
4      8080     1.0         0.0   0.0  ...      8        1680            0.0   

  yr_built  yr_renovated  zipcode      lat     long  sqft_living15  sqft_lot15  
0     1955           0.0    98178  47.5112 -122.257           1340        5650  
1     1951        1991.0    98125  47.7210 -122.319           1690        7639  
2     1933           NaN    98028  47.7379 -122.233           2720        8062  
3     1965           0.0    98136  47.5208 -122.393           1360        5000  
4     1987           0.0    98074  47.6168 -122.045           1800        7503  

[5 rows x 21 columns]

In [191]:
df_alt = df.drop(['id', 'date', 'yr_built', 'yr_renovated', 'lat', 'long'], axis = 1)
month_list = []
sqft_basement_list = []
yr_renovated = []
for date in df['date'].values:
    month_list.append(make_month(date))
for num in df['sqft_basement'].values:
    if num == '?':
        sqft_basement_list.append(np.nan)
    else:
        sqft_basement_list.append(float(num))
for data in df['yr_renovated'].values:
    if data == 0:
        yr_renovated.append(0)
    elif data == np.nan:
        yr_renovated.append(np.nan)
    else:
        yr_renovated.append(1)
df['month'] = month_list
df_alt['month'] = month_list
df['sqft_basement'] = sqft_basement_list
df_alt['sqft_basement'] = sqft_basement_list
df['renovated'] = yr_renovated
df_alt['renovated'] = yr_renovated
df_alt['zipcode'] = df_alt['zipcode'].apply(str)
df['age_of_house'] = 2021 - df['yr_built']
df_alt['age_of_house'] = 2021 - df['yr_built']
df['total_sq_ft'] = df['sqft_above'] + df['sqft_basement']
df_alt['total_sq_ft'] = df['sqft_above'] + df['sqft_basement']
df_alt.head()

price  bedrooms  bathrooms  sqft_living  sqft_lot  floors  waterfront  \
0  221900.0         3       1.00         1180      5650     1.0         NaN   
1  538000.0         3       2.25         2570      7242     2.0         0.0   
2  180000.0         2       1.00          770     10000     1.0         0.0   
3  604000.0         4       3.00         1960      5000     1.0         0.0   
4  510000.0         3       2.00         1680      8080     1.0         0.0   

   view  condition  grade  sqft_above  sqft_basement zipcode  sqft_living15  \
0   0.0          3      7        1180            0.0   98178           1340   
1   0.0          3      7        2170          400.0   98125           1690   
2   0.0          3      6         770            0.0   98028           2720   
3   0.0          5      7        1050          910.0   98136           1360   
4   0.0          3      8        1680            0.0   98074           1800   

   sqft_lot15 month  renovated  age_of_house  total_sq_ft  
0        5650   Oct          0            66       1180.0  
1        7639   Dec          1            70       2570.0  
2        8062   Feb          1            88        770.0  
3        5000   Dec          0            56       1960.0  
4        7503   Feb          0            34       1680.0

In [163]:
cont_var = ['price', 'sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement']
cont_var_cont = ['sqft_living15', 'sqft_lot15']
categorical_var = ['waterfront', 'view', 'condition', 'floors', 'bedrooms']
categorical_var_cont = ['bathrooms', 'grade', 'zipcode']

In [5]:
df.isnull().sum()

id                  0
date                0
price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       2376
view               63
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
yr_renovated     3842
zipcode             0
lat                 0
long                0
sqft_living15       0
sqft_lot15          0
dtype: int64

In [12]:
df['age_of_house'] = 2021 - df['yr_built']

In [14]:
df.corr()

id     price  bedrooms  bathrooms  sqft_living  sqft_lot  \
id             1.000000 -0.016772  0.001150   0.005162    -0.012241 -0.131911   
price         -0.016772  1.000000  0.308787   0.525906     0.701917  0.089876   
bedrooms       0.001150  0.308787  1.000000   0.514508     0.578212  0.032471   
bathrooms      0.005162  0.525906  0.514508   1.000000     0.755758  0.088373   
sqft_living   -0.012241  0.701917  0.578212   0.755758     1.000000  0.173453   
sqft_lot      -0.131911  0.089876  0.032471   0.088373     0.173453  1.000000   
floors         0.018608  0.256804  0.177944   0.502582     0.353953 -0.004814   
waterfront    -0.004176  0.276295 -0.002386   0.067282     0.110230  0.023143   
view           0.011592  0.395734  0.078523   0.186451     0.282532  0.075298   
condition     -0.023803  0.036056  0.026496  -0.126479    -0.059445 -0.008830   
grade          0.008188  0.667951  0.356563   0.665838     0.762779  0.114731   
sqft_above    -0.010799  0.605368  0.479386   0.686668     0.876448  0.184139   
yr_built       0.021617  0.053953  0.155670   0.507173     0.318152  0.052946   
yr_renovated  -0.012010  0.129599  0.018495   0.051050     0.055660  0.004513   
zipcode       -0.008211 -0.053402 -0.154092  -0.204786    -0.199802 -0.129586   
lat           -0.001798  0.306692 -0.009951   0.024280     0.052155 -0.085514   
long           0.020672  0.022036  0.132054   0.224903     0.241214  0.230227   
sqft_living15 -0.002701  0.585241  0.393406   0.569884     0.756402  0.144763   
sqft_lot15    -0.138557  0.082845  0.030690   0.088303     0.184342  0.718204   
age_of_house  -0.021617 -0.053953 -0.155670  -0.507173    -0.318152 -0.052946   

                 floors  waterfront      view  condition     grade  \
id             0.018608   -0.004176  0.011592  -0.023803  0.008188   
price          0.256804    0.276295  0.395734   0.036056  0.667951   
bedrooms       0.177944   -0.002386  0.078523   0.026496  0.356563   
bathrooms      0.502582    0.067282  0.186451  -0.126479  0.665838   
sqft_living    0.353953    0.110230  0.282532  -0.059445  0.762779   
sqft_lot      -0.004814    0.023143  0.075298  -0.008830  0.114731   
floors         1.000000    0.021883  0.028436  -0.264075  0.458794   
waterfront     0.021883    1.000000  0.406654   0.017642  0.087383   
view           0.028436    0.406654  1.000000   0.045735  0.249727   
condition     -0.264075    0.017642  0.045735   1.000000 -0.146896   
grade          0.458794    0.087383  0.249727  -0.146896  1.000000   
sqft_above     0.523989    0.075463  0.166299  -0.158904  0.756073   
yr_built       0.489193   -0.026079 -0.054564  -0.361592  0.447865   
yr_renovated   0.003535    0.087244  0.100964  -0.061858  0.016669   
zipcode       -0.059541    0.031057  0.085277   0.002888 -0.185771   
lat            0.049239   -0.012772  0.006141  -0.015102  0.113575   
long           0.125943   -0.039864 -0.077894  -0.105877  0.200341   
sqft_living15  0.280102    0.088860  0.279561  -0.093072  0.713867   
sqft_lot15    -0.010722    0.032002  0.073332  -0.003126  0.120981   
age_of_house  -0.489193    0.026079  0.054564   0.361592 -0.447865   

               sqft_above  yr_built  yr_renovated   zipcode       lat  \
id              -0.010799  0.021617     -0.012010 -0.008211 -0.001798   
price            0.605368  0.053953      0.129599 -0.053402  0.306692   
bedrooms         0.479386  0.155670      0.018495 -0.154092 -0.009951   
bathrooms        0.686668  0.507173      0.051050 -0.204786  0.024280   
sqft_living      0.876448  0.318152      0.055660 -0.199802  0.052155   
sqft_lot         0.184139  0.052946      0.004513 -0.129586 -0.085514   
floors           0.523989  0.489193      0.003535 -0.059541  0.049239   
waterfront       0.075463 -0.026079      0.087244  0.031057 -0.012772   
view             0.166299 -0.054564      0.100964  0.085277  0.006141   
condition       -0.158904 -0.361592     -0.061858  0.002888 -0.015102   
grade            0.756073  0.447865      0.01

In [22]:
df_c = df.corr().abs().stack().reset_index().sort_values(0, ascending=False)
df_c['pairs'] = list(zip(df_c.level_0, df_c.level_1))
df_c.set_index(['pairs'], inplace = True)
df_c.drop(columns=['level_1', 'level_0'], inplace = True)
df_c.columns = ['cc']
df_c.drop_duplicates(inplace=True)
df_c[(df_c.cc>.75) & (df_c.cc <1)]


cc
pairs                                 
(sqft_above, sqft_living)     0.876448
(sqft_living, grade)          0.762779
(sqft_living, sqft_living15)  0.756402
(grade, sqft_above)           0.756073
(bathrooms, sqft_living)      0.755758

In [29]:
x = df['price']
y = df['sqft_above']
z = df['sqft_living']
print(np.corrcoef(x, y))
print(np.corrcoef(x, z)) #We should drop sqft_above

[[1.         0.60536794]
 [0.60536794 1.        ]]
[[1.        0.7019173]
 [0.7019173 1.       ]]


In [32]:
y1 = df['grade']
print(np.corrcoef(x, y1))
print(np.corrcoef(x, z)) #We should drop grade

[[1.         0.66795077]
 [0.66795077 1.        ]]
[[1.        0.7019173]
 [0.7019173 1.       ]]


In [33]:
y2 = df['sqft_living15']
print(np.corrcoef(x, y2))
print(np.corrcoef(x, z)) #We should drop sqft_living15

[[1.        0.5852412]
 [0.5852412 1.       ]]
[[1.        0.7019173]
 [0.7019173 1.       ]]


In [34]:
print(np.corrcoef(x, y1))
print(np.corrcoef(x, y))#We should drop sqft_above

[[1.         0.66795077]
 [0.66795077 1.        ]]
[[1.         0.60536794]
 [0.60536794 1.        ]]


In [35]:
y3 = df['bathrooms']
print(np.corrcoef(x, y3))
print(np.corrcoef(x, z))#We should drop bathrooms

[[1.         0.52590562]
 [0.52590562 1.        ]]
[[1.        0.7019173]
 [0.7019173 1.       ]]


In [203]:
df_alt['bedrooms']

0        3
1        3
2        2
3        4
4        3
        ..
21592    3
21593    4
21594    2
21595    3
21596    2
Name: bedrooms, Length: 21597, dtype: int64

In [192]:
df_damn = df_alt.corr().abs().stack().reset_index().sort_values(0, ascending=False)
df_damn['pairs'] = list(zip(df_damn.level_0, df_damn.level_1))
df_damn.set_index(['pairs'], inplace = True)
df_damn.drop(columns=['level_1', 'level_0'], inplace = True)
df_damn.columns = ['cc']
df_damn.drop_duplicates(inplace=True)
df_damn[(df_damn.cc>.75) & (df_damn.cc <1)]

cc
pairs                                 
(sqft_above, total_sq_ft)     0.876678
(sqft_living, sqft_above)     0.876448
(grade, total_sq_ft)          0.763101
(grade, sqft_living)          0.762779
(sqft_living, sqft_living15)  0.756402
(sqft_living15, total_sq_ft)  0.756389
(grade, sqft_above)           0.756073
(bathrooms, sqft_living)      0.755758
(bathrooms, total_sq_ft)      0.755278

In [169]:
predictors = '+'.join(list(df_alt.columns[1:]))
predictors

'bedrooms+bathrooms+sqft_living+sqft_lot+floors+waterfront+view+condition+grade+sqft_above+sqft_basement+zipcode+sqft_living15+sqft_lot15+month+renovated+age_of_house+total_sq_ft'

In [ ]:
cont_var = ['price', 'sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement']
cont_var_cont = ['sqft_living15', 'sqft_lot15', 'age_of_household', 'total_sq_ft']
categorical_var = ['waterfront', 'view', 'condition', 'floors', 'bedrooms']
categorical_var_cont = ['bathrooms', 'grade', 'zipcode', 'renovated']

In [208]:
try_pred = 'C(bedrooms)+sqft_living+sqft_lot+C(floors)+C(waterfront)+C(view)+C(condition)+sqft_basement+sqft_lot15+C(renovated)+age_of_house+total_sq_ft'
time_pred = 'C(bedrooms)+sqft_living+sqft_lot+C(floors)+C(waterfront)+C(view)+C(condition)+sqft_basement+sqft_lot15+C(renovated)+age_of_house+total_sq_ft + C(month) + C(zipcode)'

In [211]:
lr = ols(formula = 'price ~' + time_pred , data = df_alt).fit()
lr.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.802
Model:                            OLS   Adj. R-squared:                  0.801
Method:                 Least Squares   F-statistic:                     680.4
Date:                Sun, 18 Apr 2021   Prob (F-statistic):               0.00
Time:                        20:18:00   Log-Likelihood:            -2.5189e+05
No. Observations:               18749   AIC:                         5.040e+05
Df Residuals:                   18637   BIC:                         5.049e+05
Df Model:                         111                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept            -2.056e+05   3.77e+04     -5.455      0.000   -2.79e+05   -1.32e+05
C(bedrooms)[T.2]     -2.722e+04   1.32e+04     -2.054      0.040   -5.32e+04   -1249.718
C(bedrooms)[T.3]     -5.536e+04   1.31e+04     -4.231      0.000    -8.1e+04   -2.97e+04
C(bedrooms)[T.4]     -9.713e+04   1.34e+04     -7.262      0.000   -1.23e+05   -7.09e+04
C(bedrooms)[T.5]     -1.193e+05   1.41e+04     -8.431      0.000   -1.47e+05   -9.15e+04
C(bedrooms)[T.6]     -1.567e+05   1.76e+04     -8.878      0.000   -1.91e+05   -1.22e+05
C(bedrooms)[T.7]     -2.948e+05   3.22e+04     -9.168      0.000   -3.58e+05   -2.32e+05
C(bedrooms)[T.8]     -1.368e+05   5.25e+04     -2.605      0.009    -2.4e+05   -3.39e+04
C(bedrooms)[T.9]     -3.949e+05   6.96e+04     -5.672      0.000   -5.31e+05   -2.58e+05
C(bedrooms)[T.10]    -4.917e+05    9.7e+04     -5.071      0.000   -6.82e+05   -3.02e+05
C(bedrooms)[T.11]    -2.264e+05   1.67e+05     -1.357      0.175   -5.53e+05    1.01e+05
C(bedrooms)[T.33]    -2.931e+04   1.67e+05     -0.176      0.860   -3.56e+05    2.97e+05
C(floors)[T.1.5]     -2.907e+04   4957.424     -5.865      0.000   -3.88e+04   -1.94e+04
C(floors)[T.2.0]     -4.292e+04   3985.366    -10.769      0.000   -5.07e+04   -3.51e+04
C(floors)[T.2.5]      5.048e+04   1.48e+04      3.401      0.001    2.14e+04    7.96e+04
C(floors)[T.3.0]     -7.353e+04   9041.908     -8.132      0.000   -9.13e+04   -5.58e+04
C(floors)[T.3.5]      2.582e+04   6.85e+04      0.377      0.706   -1.09e+05     1.6e+05
C(waterfront)[T.1.0]  5.962e+05   1.78e+04     33.471      0.000    5.61e+05    6.31e+05
C(view)[T.1.0]        9.302e+04   1.01e+04      9.237      0.000    7.33e+04    1.13e+05
C(view)[T.2.0]        8.664e+04   6199.885     13.975      0.000    7.45e+04    9.88e+04
C(view)[T.3.0]        1.797e+05   8522.706     21.082      0.000    1.63e+05    1.96e+05
C(view)[T.4.0]        3.519e+05   1.27e+04     27.637      0.000    3.27e+05    3.77e+05
C(condition)[T.2]     5.765e+04   3.66e+04      1.574      0.115   -1.41e+04    1.29e+05
C(condition)[T.3]     6.405e+04   3.41e+04      1.878      0.060   -2814.864    1.31e+05
C(condition)[T.4]     7.865e+04   3.41e+04      2.304      0.021    1.18e+04    1.46e+05
C(condition)[T.5]     1.272e+05   3.43e+04      3.705      0.000    5.99e+04    1.94e+05
C(renovated)[T.1]     8371.7315   3009.377      2.782      0.005    2473.078    1.43e+04
C(month)[T.Aug]      -3.649e+04   5519.836     -6.610      0.000   -4.73e+04   -2.57e+04
C(month)[T.Dec]      -3.673e+04   5970.266     -6.152      0.000   -4.84e+04    -2.5e+04
C(month)[T.Feb]      -2.693e+04   6331.229     -4.253      0.000   -3.93e+04   -1.45e+04
C(month)[T.Jan]      -3.494e+04   6861.307     -5.092      0.000   -4.84e+04   -2.15e+04
C(month)[T.Jul]       -3.87e+04   5345.336     -7.239    